<a id='installing-spark'></a>
### Installing Spark

Install Dependencies:


1.   Java 8
2.   Apache Spark with hadoop and
3.   Findspark (used to locate the spark in the system)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Set Environment Variables:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
# sc = SparkContext()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We are going to download some datasets called MovieLens
https://grouplens.org/datasets/movielens/

In [ ]:
data = spark.read.csv('C:\Users\AI\Desktop\ml-25m\movies.csv',inferSchema=True, header=True)

NameError: ignored

In [ ]:
sc = SparkContext.getOrCreate();
from pyspark.sql import SparkSession
spark = SparkSession(sc)
data = spark.read.csv("/content/drive/MyDrive/movies.csv",inferSchema=True, header=True)

In [ ]:
data.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

Can you find how many movies are there in movies.csv?  

In [ ]:
data.count()

62423

Is movieID all unique?

In [ ]:
data.movieId

Column<'movieId'>

In [ ]:
data.dropDuplicates(['movieId'])

DataFrame[movieId: int, title: string, genres: string]

In [ ]:
dedupe = data.dropDuplicates(['movieId'])
dedupe.count()

62423

Find min and max of movieID

In [ ]:
dedupe.movieId

Column<'movieId'>

In [ ]:
from pyspark.sql.functions import min
data.select([min("movieId")]).show()

+------------+
|min(movieId)|
+------------+
|           1|
+------------+



In [ ]:
data.select("movieId").rdd.min()

Row(movieId=1)

In [ ]:
data.select("movieId").rdd.min()[0]

1

For each movie, count the number of genres

In [ ]:
rdd = data.select("genres").rdd
rdd.collect()

#.rdd.map(lambda x: f(x[0])).collect()

[Row(genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(genres='Adventure|Children|Fantasy'),
 Row(genres='Comedy|Romance'),
 Row(genres='Comedy|Drama|Romance'),
 Row(genres='Comedy'),
 Row(genres='Action|Crime|Thriller'),
 Row(genres='Comedy|Romance'),
 Row(genres='Adventure|Children'),
 Row(genres='Action'),
 Row(genres='Action|Adventure|Thriller'),
 Row(genres='Comedy|Drama|Romance'),
 Row(genres='Comedy|Horror'),
 Row(genres='Adventure|Animation|Children'),
 Row(genres='Drama'),
 Row(genres='Action|Adventure|Romance'),
 Row(genres='Crime|Drama'),
 Row(genres='Drama|Romance'),
 Row(genres='Comedy'),
 Row(genres='Comedy'),
 Row(genres='Action|Comedy|Crime|Drama|Thriller'),
 Row(genres='Comedy|Crime|Thriller'),
 Row(genres='Crime|Drama|Horror|Mystery|Thriller'),
 Row(genres='Action|Crime|Thriller'),
 Row(genres='Drama|Sci-Fi'),
 Row(genres='Drama|Romance'),
 Row(genres='Drama'),
 Row(genres='Children|Drama'),
 Row(genres='Drama|Romance'),
 Row(genres='Adventure|Drama|Fantasy|

In [ ]:
rdd2 = rdd.map(lambda x: x[0])
rdd2.collect()

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy',
 'Action|Crime|Thriller',
 'Comedy|Romance',
 'Adventure|Children',
 'Action',
 'Action|Adventure|Thriller',
 'Comedy|Drama|Romance',
 'Comedy|Horror',
 'Adventure|Animation|Children',
 'Drama',
 'Action|Adventure|Romance',
 'Crime|Drama',
 'Drama|Romance',
 'Comedy',
 'Comedy',
 'Action|Comedy|Crime|Drama|Thriller',
 'Comedy|Crime|Thriller',
 'Crime|Drama|Horror|Mystery|Thriller',
 'Action|Crime|Thriller',
 'Drama|Sci-Fi',
 'Drama|Romance',
 'Drama',
 'Children|Drama',
 'Drama|Romance',
 'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
 'Crime|Drama',
 'Drama',
 'Mystery|Sci-Fi|Thriller',
 'Adventure|Romance|IMAX',
 'Children|Drama',
 'Drama|Romance',
 'Crime|Drama',
 'Documentary|IMAX',
 'Children|Comedy',
 'Comedy|Romance',
 'Drama',
 'Drama|War',
 'Action|Crime|Drama',
 'Drama',
 'Action|Adventure|Fantasy',
 'Comedy|Drama|Thriller',
 'Drama|Romance',
 'Myst

Any thoughts?

> Let's first do some testings



In [ ]:
str = "Children|Drama|Fantasy|Romance"

In [ ]:
str.split('|')

['Children', 'Drama', 'Fantasy', 'Romance']

In [ ]:
rdd3 = rdd.map(lambda x: len(x[0].split('|')))
rdd3.collect()

[5,
 3,
 2,
 3,
 1,
 3,
 2,
 2,
 1,
 3,
 3,
 2,
 3,
 1,
 3,
 2,
 2,
 1,
 1,
 5,
 3,
 5,
 3,
 2,
 2,
 1,
 2,
 2,
 5,
 2,
 1,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 3,
 1,
 3,
 3,
 2,
 2,
 5,
 2,
 3,
 3,
 3,
 2,
 2,
 1,
 4,
 1,
 3,
 2,
 3,
 2,
 1,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 4,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 2,
 1,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 1,
 2,
 4,
 1,
 3,
 1,
 2,
 2,
 4,
 1,
 3,
 3,
 3,
 4,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 3,
 1,
 2,
 3,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 5,
 2,
 1,
 1,
 3,
 3,
 4,
 2,
 4,
 1,
 3,
 2,
 2,
 2,
 3,
 4,
 3,
 1,
 3,
 4,
 3,
 3,
 1,
 3,
 3,
 4,
 2,
 3,
 3,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 2,
 1,
 1,
 2,
 3,
 2,
 1,
 6,
 3,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 3,
 1,
 1,
 1,
 4,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 3,
 1,
 1,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 2,
 3,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,


Append a column which is the counting of genres

In [ ]:
rdd3.toDF(["count_genres"])

TypeError: ignored

In [ ]:
from pyspark.sql import Row
rdd3.map(lambda x: Row(x)).toDF(["count_genres"]).show()

+------------+
|count_genres|
+------------+
|           5|
|           3|
|           2|
|           3|
|           1|
|           3|
|           2|
|           2|
|           1|
|           3|
|           3|
|           2|
|           3|
|           1|
|           3|
|           2|
|           2|
|           1|
|           1|
|           5|
+------------+
only showing top 20 rows



In [ ]:
data.count_genres = rdd3.map(lambda x: Row(x)).toDF(["count_genres"])
data.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [ ]:
data.rdd

MapPartitionsRDD[216] at javaToPython at NativeMethodAccessorImpl.java:0

Above doesn't work...



In [ ]:

def customColumn(row):
    rD=row.asDict()
    rD["countGenres"]= len( row["genres"].split('|') )
    #rD["countGenres"]= len(row["title"])

    new_row=Row(**rD)
    return new_row

datardd= data.rdd
outputDF=datardd.map(customColumn).toDF()
outputDF.show()

+-------+--------------------+--------------------+-----------+
|movieId|               title|              genres|countGenres|
+-------+--------------------+--------------------+-----------+
|      1|    Toy Story (1995)|Adventure|Animati...|          5|
|      2|      Jumanji (1995)|Adventure|Childre...|          3|
|      3|Grumpier Old Men ...|      Comedy|Romance|          2|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|          3|
|      5|Father of the Bri...|              Comedy|          1|
|      6|         Heat (1995)|Action|Crime|Thri...|          3|
|      7|      Sabrina (1995)|      Comedy|Romance|          2|
|      8| Tom and Huck (1995)|  Adventure|Children|          2|
|      9| Sudden Death (1995)|              Action|          1|
|     10|    GoldenEye (1995)|Action|Adventure|...|          3|
|     11|American Presiden...|Comedy|Drama|Romance|          3|
|     12|Dracula: Dead and...|       Comedy|Horror|          2|
|     13|        Balto (1995)|Adventure|

**stil looking for another way? **

Let's use UDF!!!

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

split_words = udf(lambda x : x.split('|') if x is not None else x, StringType())
data = data.withColumn('countGenres', split_words(data['genres']))
data.show()

+-------+--------------------+--------------------+--------------------+
|movieId|               title|              genres|         countGenres|
+-------+--------------------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|[Adventure, Anima...|
|      2|      Jumanji (1995)|Adventure|Childre...|[Adventure, Child...|
|      3|Grumpier Old Men ...|      Comedy|Romance|   [Comedy, Romance]|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|[Comedy, Drama, R...|
|      5|Father of the Bri...|              Comedy|            [Comedy]|
|      6|         Heat (1995)|Action|Crime|Thri...|[Action, Crime, T...|
|      7|      Sabrina (1995)|      Comedy|Romance|   [Comedy, Romance]|
|      8| Tom and Huck (1995)|  Adventure|Children|[Adventure, Child...|
|      9| Sudden Death (1995)|              Action|            [Action]|
|     10|    GoldenEye (1995)|Action|Adventure|...|[Action, Adventur...|
|     11|American Presiden...|Comedy|Drama|Romance|

Problem (difficult)  Count the occurrence of each genre

**Now Let's read ratings data**

In [ ]:
ratings = spark.read.csv('/content/drive/MyDrive/ratings.csv',inferSchema=True, header=True)

In [ ]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:
ratings.groupby(['userId', 'movieId']).count().where('count > 1').sort('count', ascending=False).show()

+------+-------+-----+
|userId|movieId|count|
+------+-------+-----+
+------+-------+-----+



Drop timestamp

In [ ]:
ratings = ratings.drop(ratings.timestamp)

In [ ]:
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows



Sort the dataframe above respect to movieId

In [ ]:
ratings.sort(ratings.movieId.asc()).show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
| 32961|      1|   0.5|
| 33030|      1|   3.5|
| 32963|      1|   4.0|
| 32909|      1|   2.5|
| 32966|      1|   4.0|
| 32914|      1|   3.0|
| 32967|      1|   4.0|
| 32922|      1|   4.0|
| 32968|      1|   4.0|
| 32927|      1|   2.5|
| 32971|      1|   5.0|
| 32934|      1|   5.0|
| 32974|      1|   4.0|
| 32938|      1|   5.0|
| 32975|      1|   4.0|
| 32941|      1|   4.0|
| 32976|      1|   5.0|
| 32945|      1|   4.0|
| 32983|      1|   2.5|
| 32951|      1|   1.0|
+------+-------+------+
only showing top 20 rows



DataFrame.rdd
Returns the content as an pyspark.RDD of Row.

In [ ]:
ratings.orderBy(["userId", "movieId"], ascending=[0, 1]).show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|162541|     29|   5.0|
|162541|     32|   5.0|
|162541|     47|   4.5|
|162541|     50|   5.0|
|162541|    105|   2.5|
|162541|    141|   2.5|
|162541|    145|   3.0|
|162541|    150|   3.5|
|162541|    153|   2.5|
|162541|    208|   1.0|
|162541|    260|   5.0|
|162541|    261|   3.5|
|162541|    293|   4.0|
|162541|    296|   3.5|
|162541|    318|   4.0|
|162541|    344|   0.5|
|162541|    345|   4.5|
|162541|    410|   1.5|
|162541|    412|   3.5|
|162541|    416|   1.5|
+------+-------+------+
only showing top 20 rows



Anything wrong with movie ID, User ID?

In [ ]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



df.withColumn("age",df.age.cast(IntegerType()))

df.withColumn("age",df.age.cast('int'))

df.withColumn("age",df.age.cast('integer'))





In [ ]:
ratings.withColumn("userId",ratings.userId.cast('integer')).show()


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
ratings = ratings.withColumn("userId",ratings.userId.cast('integer'))
ratings = ratings.withColumn("movieId",ratings.movieId.cast('integer'))

In [ ]:
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
ratings.count()

25000095

In [ ]:
ratings.na.drop().count()

25000095